### Load ENV and LLM

In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")

In [2]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_tavily import TavilySearch

arxiv_api = ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wiki_api = WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)

arxiv = ArxivQueryRun(api_wrapper=arxiv_api)
wiki = WikipediaQueryRun(api_wrapper=wiki_api)
tavily = TavilySearch()


D:\agentic-ai-bootcamp\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
D:\agentic-ai-bootcamp\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


### Create a Custom Tools

In [3]:
def add(a:int,b:int)->int:
    return a+b

def multiply(a:int,b:int)->int:
    return a*b

def divide(a:int,b:int)->int:
    return a/b

In [4]:
### Add all tools in a list formate
tools = [arxiv,wiki,tavily,add,multiply,divide]

### Bind Tools

In [5]:
model_with_tools = model.bind_tools(tools)

### Define the State`

In [6]:
from typing import TypedDict,Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    messages:Annotated[list,add_messages]

### Define the Nodes

In [1]:
def llm_call(state:State)->State:
    """Call the correct tool based on the state"""
    print("LLM Call node has been called")
    return {"messages"[model_with_tools.invoke(state["messages"])]}

NameError: name 'State' is not defined

### Create the Graph Structure

In [ ]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode,tools_condition

#Define the Graph
graph = StateGraph(State)

## Add the Nodes
graph.add_node("llm_call",llm_call)
graph.add_node("tools",ToolNode(tools))

##Add Edges
graph.add_edge(START,"llm_call")
graph.add_edge("llm_call",tools_condition("tools"))
graph.add_edge("llm_call",END)

graph_builder = graph.compile()

display(Image(graph_builder.get_graph().draw_mermaid_png()))
